# Inference a fine-tuned model with function calling - token reduction scenario

## Objective

In this notebook, we'll demonstrate how to inference our fine-tuned model with function calling for the token reduction scenario. 

After your fine-tuned model is deployed, you can use it like any other deployed model via the chat completion API. 


## Time

You should expect to sepnd 5-10 min running this sample.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

In [ ]:
# Install the packages\
%pip install requests openai~=1.10

Import the required packages.

In [12]:
import os
import json
import pandas as pd
from openai import AzureOpenAI

## Run this example

Define gpt_test function for inference and initialize an Azure OpenAI client for interaction with the Azure OpenAI service.

In [17]:
def gpt_test():
    print('gpt_inference')

client = AzureOpenAI(
    #azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_endpoint="https://<YOUR_RESOURCE_NAME>.openai.azure.com",
    api_key= "<AOAI_RESOURCE_API_KEY>", 
    api_version="2023-07-01-preview",
)


Lets declare function schema. This schema can contain multiple functions which can perform multiple intents. Our stock use case features two functions: the first one retrieves the current stock price, and the second one gets the stock price of last n days. 

This is the shortened version of the function where descriptions and properties from the parameters object have been removed.

In [25]:
functions = [
            {
                "name": "get_current_stock_price",
                "parameters": {
                    "type": "object",
                    "properties": {
                    },
                    "required": [
                        "symbol"
                    ]
                }
            },
            {
                "name": "get_last_nday_stock_price",
            "parameters": {
                "type": "object",
                "properties": {
                },
                "required": ["symbol", "period"]
                }
            }
        ]

lets add the content of our test dataset (stock-test-token-reduction.jsonl) into a list of strings.

In [19]:
with open('Data/stock-test-token-reduction.jsonl', errors='ignore') as json_file:
    json_list = list(json_file)

We need to define the message with the question that we want to ask to gpt.

This code extracts system and user content from the JSON string, creates a list of messages to be sent to the Azure OpenAI model for completion, sends messages for completion and finally prints the model's completion response along with any errors encountered.

we set tempretre as 0 to reduce randomness.

In [ ]:
mismatch_count = 0
for i, json_str in enumerate(json_list[:1]):
    print('starting on ', i)
    result = json.loads(json_str)
    if len(result['messages']) > 2:
        system_content = result['messages'][0]['content']
        user_content = result['messages'][1]['content']
    else:
        user_content = result['messages'][0]['content']

    messages = [
                    {"role": "system", "content": system_content},
                    #{"role": "user", "content": user_content},
                    {"role": "user", "content": "what is the current price of Uber?"}, 
                    #{"role": "user", "content": "What was the highest price that walmart's stock reached last quarter?"},
                ]
 
    try:
        completion = client.chat.completions.create(
            model="<DEPLOYMENT_NAME>", # add the fine-tuned model deployment name
            messages=messages,
            temperature=0.0,  # to reduce randomness
            functions=functions,
            function_call="auto",
        )
        try:
            response_message = completion.choices[0].message
            print(completion.choices[0].message.model_dump_json(indent=2))


        except Exception as e:
            print('Error', i, completion)
            print(e)

    except Exception as e:
        print('Error', i)
        print(e)


if __name__ == '__main__':
    gpt_test()